# 实验1，

测试chunk length, 还有目标文本在chunk中的位置对得分的影响。假设有一个目标paragraph tp, n个不相关的 np. 实验主要是通过拼接不同的text，把目标文档放不同位置，计算得分，做对比。

单独一个tp得分, t1.

tp放最前面，后面接n个不同的np，形成一个chunk，得分t2.

tp放中间，前面放n个不同的np,后面放n个不同的np，得分t2.

tp放文档末尾，前面放n个不同的np, 得分t3.

In [ ]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"  # 国内用户加速模型下载
os.environ["HF_ENDPOINT"] = "https://huggingface.byteintl.com"  # 国内用户加速模型下载


# 1. 导入依赖库
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 2. 加载 bge-m3 模型（自动下载缓存，国内可配置镜像）
model = SentenceTransformer("baai/bge-m3")
model.max_seq_length = 5120  # 设置文本最大长度

# 1. 加载模型（自动下载并缓存）
model = SentenceTransformer("baai/bge-m3")
model.max_seq_length = 5120  # 设置最大文本长度

# 2. 生成 Embedding（支持单条/批量，默认归一化）
text1 = "Elasticsearch 向量搜索使用 HNSW 算法"
embedding1 = model.encode(text1, normalize_embeddings=True)
print(f"单条文本 Embedding 维度：{embedding1.shape}")  # (1024,)

# 批量生成
query="speed of light"
texts = [
    "Science is a powerful tool that allows humans to observe, understand, and explain the world around them. Through centuries of inquiry and experimentation, scientists have uncovered countless facts about the universe, many of which shape the way we live today. Below are some fascinating scientific facts that highlight the beauty and complexity of nature.",
    """# The Universe Is Expanding

One of the most profound discoveries in modern astronomy is that the universe is constantly expanding. In 1929, Edwin Hubble observed that galaxies are moving away from each other, implying that the universe had a beginning. This discovery laid the foundation for the Big Bang theory, which suggests that the cosmos began around 13.8 billion years ago.
""",
    """# DNA: The Blueprint of Life
Deoxyribonucleic acid (DNA) contains the genetic instructions for building and maintaining life. Every living organism on Earth shares this molecular code, though arranged differently. A surprising fact is that humans share about 60% of their DNA with bananas, highlighting the interconnectedness of life on Earth.
""",
    """# The Human Brain’s Complexity
The human brain is considered the most complex structure known in the universe. It contains around 86 billion neurons, each connected to thousands of others, creating trillions of synaptic connections. These networks enable thought, memory, emotions, and consciousness. Remarkably, the brain uses about 20% of the body’s energy, even though it accounts for only 2% of body mass.""",
    """# Water Is Unique
Water has extraordinary properties that make life possible. Unlike most substances, water expands when it freezes, causing ice to float. This prevents bodies of water from freezing solid and allows aquatic life to survive under ice sheets. Water is also a universal solvent, enabling countless chemical reactions within cells.""",
    """# The Speed of Light
Light travels at approximately 299,792 kilometers per second (186,282 miles per second) in a vacuum. To put this into perspective, light from the Sun takes just over eight minutes to reach Earth. This incredible speed makes light the fastest thing in the universe and serves as a fundamental constant in physics.""",
    """# The Earth’s Protective Atmosphere
Earth’s atmosphere shields us from harmful radiation and space debris. The ozone layer, in particular, absorbs most of the Sun’s ultraviolet rays, protecting living organisms from genetic damage. Without the atmosphere, life as we know it would not exist.""",
    """# Evolution Shapes Life
Charles Darwin’s theory of evolution by natural selection remains one of the most important scientific discoveries. Over millions of years, species adapt to their environments, resulting in the incredible diversity of life we see today. For example, whales evolved from land-dwelling mammals that returned to the sea about 50 million years ago.""",
    """# Conclusion
Scientific facts remind us that the universe is vast, life is interconnected, and human knowledge continues to grow. From the tiniest molecule of DNA to the largest galaxies, science reveals the underlying order and beauty of existence. As we uncover more truths, our appreciation of the natural world deepens, inspiring innovation, curiosity, and respect for life."""
]

chunk1 = texts[5] + texts[:5] + texts[6:] # 目标文档在头部
chunk2 = texts[:5] + texts[5]  + texts[6:] # 目标段落在中间
chunk3
query_embedding = model.encode(query, normalize_embeddings=True).reshape(1, -1)  # (1, 1024)
texts_embeddings = model.encode(texts, normalize_embeddings=True)  # (n, 1024)，n 是 texts 长度

print(f"批量 Embedding 形状：{texts_embeddings.shape}")  # (10, 1024)

/Users/AndrewYang/sourcecode/rag_research/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.byteintl.com\', port=443): Max retries exceeded with url: /baai/bge-m3/resolve/main/modules.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x31515bb60>: Failed to resolve \'huggingface.byteintl.com\' ([Errno 8] nodename nor servname provided, or not known)"))'), '(Request ID: 7d517d2a-0218-483e-9aeb-a045417f7e86)')' thrown while requesting HEAD https://huggingface.byteintl.com/baai/bge-m3/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.byteintl.com\', port=443): Max retries exceeded with url: /baai/bge-m3/resolve/main/modules.json (Caused by NameResolution

KeyboardInterrupt: 